## Estimation of Breeding Values

A notebook of Lynch & Walsh (1996), chapter 26. This notebook only takes the example part. Or, it mainly serves as another illustration of Julia usage.

For the general mixed model:

$\mathbf{y}_{n\times 1} = \mathbf{X}_{n\times p}\beta_{p\times 1}+\mathbf{Z}_{n\times q}\mathbf{u}_{q\times 1}+\mathbf{e}_{n\times 1}$

$\mathbf{X}_{n\times p}$ and $\mathbf{Z}_{n\times q}$ are incidence matrices. $\mathbf{X}_{n\times p}$ is also called design matrix.

### Example 1
| Daughter | Sire | Environment | Phenotype|
| :--: | --: | :--: | --: |
| 1 | 1 | 1 | 9 |
| 2 | 1 | 2 | 12 |
| 3 | 2 | 1 | 11 |
| 4 | 2 | 1 | 6 |
| 5 | 3 | 1 | 7 |
| 6 | 3 | 2 | 14 |

We will use `DataFrames` package to deal with the raw data.

In [1]:
using DataFrames, LinearAlgebra

In [7]:
df = DataFrame(daughter=1:6, 
    sire = [1, 1, 2, 2, 3, 3], 
    environment=[1,2,1,1,1,2], 
    phenotype = [9, 12, 11, 6, 7, 14])

,daughter,sire,environment,phenotype
,Int64,Int64,Int64,Int64
1,1,1,1,9
2,2,1,2,12
3,3,2,1,11
4,4,2,1,6
5,5,3,1,7
6,6,3,2,14


Because the data is very simple, we can directly write the incidence matrices, e.g.,
$X=\begin{bmatrix}1 & 0\\0 & 1\\1 & 0\\1 & 0\\1 & 0\\0 & 1\end{bmatrix}$. The construction of an incidence matrix can be generalized with a function. We will put it in the end of this notebook to avoid to get into too much details in the beginning.

In [5]:
X = [1 0
    0 1
    1 0
    1 0
    1 0
    0 1]

6×2 Matrix{Int64}:
 1  0
 0  1
 1  0
 1  0
 1  0
 0  1

In [6]:
Z = [ 1 0 0
    1 0 0
    0 1 0
    0 1 0
    0 0 1
    0 0 1]

6×3 Matrix{Int64}:
 1  0  0
 1  0  0
 0  1  0
 0  1  0
 0  0  1
 0  0  1

And we have effects that are to be estimated/predicted. $\beta=\begin{bmatrix}\beta_1\\\beta_2\end{bmatrix}$, and $\mathbf{u}=\begin{bmatrix}u_1\\ u_2\\ u_3\end{bmatrix}$.

The brute force way to estimation and prediction of $\beta$ and $\mathbf{u}$ in a mixed model are:

$\hat{\beta} = \left(\mathbf{X}'\mathbf{V}^{-1}\mathbf{X}\right)^{-1}\mathbf{X}'\mathbf{V}^{-1}\mathbf{y}$

and

$\hat{\mathbf{u}}=\mathbf{GZ}'\mathbf{V}^{-1}(\mathbf{y-X}\hat{\beta})$

It is assummed that the variance components are known in this chapter.

$\mathbf{R}=\sigma_E^2\mathbf{I}$,

and 

$\mathbf{G} = \sigma_S^2\mathbf{I}=\frac{\sigma_A^2}4\mathbf{I}$

We have $\sigma_E^2=6$, and $\sigma_A^2=8$

In [8]:
G = 8/4 * I

UniformScaling{Float64}
2.0*I

In [9]:
R = 6I

UniformScaling{Int64}
6*I

In [10]:
V = Z * G * Z' + R

6×6 Matrix{Float64}:
 8.0  2.0  0.0  0.0  0.0  0.0
 2.0  8.0  0.0  0.0  0.0  0.0
 0.0  0.0  8.0  2.0  0.0  0.0
 0.0  0.0  2.0  8.0  0.0  0.0
 0.0  0.0  0.0  0.0  8.0  2.0
 0.0  0.0  0.0  0.0  2.0  8.0

In [11]:
Vⁱ = inv(V)

6×6 Matrix{Float64}:
  0.133333   -0.0333333   0.0        -0.0         0.0        -0.0
 -0.0333333   0.133333    0.0        -0.0         0.0        -0.0
  0.0         0.0         0.133333   -0.0333333   0.0        -0.0
  0.0         0.0        -0.0333333   0.133333    0.0        -0.0
  0.0         0.0         0.0         0.0         0.133333   -0.0333333
  0.0         0.0         0.0         0.0        -0.0333333   0.133333

In [14]:
y = [9, 12, 11, 6, 7, 14]

6-element Vector{Int64}:
  9
 12
 11
  6
  7
 14

In [15]:
bhat = (X'Vⁱ * X) \ (X'Vⁱ * y)

2-element Vector{Float64}:
  8.222222222222221
 13.055555555555557

In [16]:
uhat = G * Z'Vⁱ*(y - X*bhat)

3-element Vector{Float64}:
 -0.05555555555555572
  0.11111111111111138
 -0.05555555555555572

### A summary of example 1

We can see that matrix manipulations very very straightforward with Julia. Before Julia 1.4, the first run of a Julia function is slow, especially plotting. It had even a special name, TTFP (time to first plot). Now this is very much improved. Since Julia 1.6, the precompilation of packages are parallized. Julia is still a fast evolving language. So are its packages. R pacakges have the same phenomena. There is a joke about R users. Most R users never update their packages after first installation.

Note, I also showed the explanation in markdown languages. To know more of markdown, you can refer my other notebook: `beforeJulia.ipynb`.

In the demonstration below, we will try to do the calculation with MME(mixed-model equations), that is:

$\begin{bmatrix}\mathbf{X}'\mathbf{R}^{-1}\mathbf{X} & \mathbf{X}'\mathbf{R}^{-1}\mathbf{Z}\\
\mathbf{Z}'\mathbf{R}^{-1}\mathbf{X} & \mathbf{Z}'\mathbf{R}^{-1}\mathbf{Z}+\mathbf{G}^{-1}\end{bmatrix}
\begin{bmatrix}\hat{\beta}\\ \hat{\mathbf{u}}\end{bmatrix} = 
\begin{bmatrix}\mathbf{X}'\mathbf{R}^{-1}\mathbf{y}\\ \mathbf{Z}'\mathbf{R}^{-1}\mathbf{y}\end{bmatrix}$

In [18]:
Rⁱ = inv(R)

UniformScaling{Float64}
0.16666666666666666*I

In [19]:
Gⁱ = inv(G)

UniformScaling{Float64}
0.5*I

In [20]:
lhs = [X'Rⁱ * X X'Rⁱ * Z
       Z'Rⁱ * X Z'Rⁱ * Z + Gⁱ]

5×5 Matrix{Float64}:
 0.666667  0.0       0.166667  0.333333  0.166667
 0.0       0.333333  0.166667  0.0       0.166667
 0.166667  0.166667  0.833333  0.0       0.0
 0.333333  0.0       0.0       0.833333  0.0
 0.166667  0.166667  0.0       0.0       0.833333

In [22]:
rhs = [X'Rⁱ * y
    Z'Rⁱ * y]

5-element Vector{Float64}:
 5.5
 4.333333333333333
 3.5
 2.833333333333333
 3.4999999999999996

In [23]:
lhs \ rhs

5-element Vector{Float64}:
  8.222222222222223
 13.055555555555557
 -0.0555555555555555
  0.11111111111111063
 -0.05555555555555602